# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Y. Wang  ->  Y. Wang  |  ['Y. Wang']
S. Shahaf  ->  S. Shahaf  |  ['S. Shahaf']
J. Li  ->  J. Li  |  ['J. Li']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
J. Li  ->  J. Li  |  ['J. Li']
Y. Wu  ->  Y. Wu  |  ['Y. Wu']
J. Li  ->  J. Li  |  ['J. Li']


X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Li  ->  J. Li  |  ['J. Li']
Arxiv has 90 new papers today
          7 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/7 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2510.17906


extracting tarball to tmp_2510.17906...

 done.
Retrieving document from  https://arxiv.org/e-print/2510.17957


extracting tarball to tmp_2510.17957...

...

extracting tarball to tmp_2510.18248...

 done.
Retrieving document from  https://arxiv.org/e-print/2510.18382


extracting tarball to tmp_2510.18382...

 done.
Retrieving document from  https://arxiv.org/e-print/2510.18531


extracting tarball to tmp_2510.18531... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.17957-b31b1b.svg)](https://arxiv.org/abs/2510.17957) | **The Silent Majority: The Interacting Post-Common-Envelope Binaries Underlying Cataclysmic Variables**  |
|| Y. M. Shani, et al. -- incl., <mark>S. Shahaf</mark>, <mark>J. Li</mark>, <mark>H.-W. Rix</mark> |
|*Appeared on*| *2025-10-22*|
|*Comments*| **|
|**Abstract**|            We analyze the orbital period distribution of post-common-envelope white-dwarf-main-sequence (WDMS) binaries by cross-matching the new spectroscopic Gaia DR3 WDMS catalog with TESS light curves, and applying a uniform periodicity search and vetting pipeline. We identify 107 periodic systems, including 74 eclipsing binaries (32 new) and 33 binaries exhibiting only sinusoidal variations. Injection-recovery tests and a forward detectability model yield a completeness-corrected distribution that is well-described by a two-component function: a log-period Gaussian peaking at $P_{\rm orb} \approx 4.1 $ h with $\sigma \approx 1.8$ h, plus a rising component that begins near $P_{\rm orb}\approx12.9$ h. We refer to this extended component as the long-period tail. It consists exclusively of detached non-interacting post-common-envelope binaries (PCEBs) that likely emerged from the common envelope and have not yet initiated mass transfer. In contrast, the short-period Gaussian is dominated by interacting or near-contact systems (including 22 known cataclysmic variables), consistent with high Roche-lobe filling factors. From the completeness-corrected distribution we infer that $29.8\%\pm4.5\%$ of the spatially unresolved WDMSs in our parent catalog are close PCEBs. Binary population synthesis models with high common-envelope efficiencies overproduce long-period systems and fail to reproduce the sharp peak, whereas lower efficiencies ($\alpha\lambda \leq 0.3$) match the peak more closely, yet still underpredict the tail. Our results hint at a large, currently under-classified reservoir of pre-cataclysmic variables and weakly accreting binaries, and provide new constraints on common-envelope physics.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.17906-b31b1b.svg)](https://arxiv.org/abs/2510.17906) | **Imaging and Polarimetric Signatures of Konoplya-Zhidenko Black Holes with Various Thick Disk**  |
|| X. Wang, <mark>Y. Wang</mark>, X.-X. Zeng |
|*Appeared on*| *2025-10-22*|
|*Comments*| *35pages,14figures*|
|**Abstract**|            We investigate the imaging properties of spherically symmetric Konoplya-Zhidenko (KZ) black holes surrounded by geometrically thick accretion flows, adopting a phenomenological radiatively inefficient accretion flow (RIAF) model and an analytical ballistic approximation accretion flow (BAAF) model. General relativistic radiative transfer is employed to compute synchrotron emission from thermal electrons and generate horizon-scale images. For the RIAF model, we analyze the dependence of image morphology on the deformation parameter, observing frequency, and flow dynamics. The photon ring and central dark region expand with increasing deformation parameter, with brightness asymmetries arising at high inclinations and depending on flow dynamics and emission anisotropy. The BAAF disk produces narrower rings and darker centers, while polarization patterns trace the brightness distribution and vary with viewing angle and deformation, revealing spacetime structure. These results demonstrate that intensity and polarization in thick-disk models provide probes of KZ black holes and near-horizon accretion physics.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.17996-b31b1b.svg)](https://arxiv.org/abs/2510.17996) | **Testing the Stellar Feedback-driven Breathing Mode in Low-mass Galaxies with Gas Kinematics**  |
|| Y. Luo, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-10-22*|
|*Comments*| *23 pages, 12 figures*|
|**Abstract**|            Hydrodynamic simulations have proposed that stellar feedback and bursty star-formation can produce dark matter cores in low-mass galaxies. A key prediction is that feedback-driven gas outflow and inflow cycles can lead to ``breathing modes'' (rapid fluctuations in the global gravitational potential) which drive correlated variations in galaxy size, kinematics, and star-formation rate. In this paper, we test the dynamical effects of feedback-driven breathing modes using a sample of 103 star-forming low-mass galaxies with stellar masses between $7.9<\rm \log M_*/M_\odot<9.6$ and $0.02<z<0.19$. We measure ionized gas velocity dispersions from H$\alpha$ emission lines and compare them to mock observations from the FIRE-2 simulations. We compare gas velocity dispersions ($\rm \sigma_{gas}$), stellar masses, and specific star-formation rates (sSFR). We find a positive correlation between gas velocity dispersion residuals at fixed stellar masses ($\rm \Delta\sigma_{gas}$) and sSFR in both data and simulations. However, the relation is tighter in FIRE-2 compared to the data. FIRE-2 produces more low-sSFR galaxies compared to our observational sample, however, the sSFR distributions agree after limiting both samples to a minimum sSFR. A deeper and more complete photometric sample further indicates that observed low-mass galaxies could span the full range of sSFR predicted in the FIRE-2 simulations. Our results support the existence of short-timescale dynamical effects driven by gas outflow and inflow cycles in low-mass galaxies and motivate additional tests of the breathing mode.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.18248-b31b1b.svg)](https://arxiv.org/abs/2510.18248) | **The ALPINE-CRISTAL-JWST Survey: Stellar and nebular dust attenuation of main-sequence galaxies at z~4-6**  |
|| A. Tsujita, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-10-22*|
|*Comments*| *Submitted to ApJ. Main text: 22 pages and 6 figures. Appendix: 1 figure. See also the companion papers by Faisst et al. and Fujimoto et al submitted on 20 Oct (Mon). Comments are welcome!*|
|**Abstract**|            Characterizing dust attenuation is crucial for revealing the intrinsic physical properties of galaxies. We present an analysis of dust attenuation in 18 spectroscopically confirmed star-forming main-sequence galaxies at $z = 4.4-5.7$ observed with JWST/NIRSpec IFU and NIRCam, selected from the ALPINE and CRISTAL ALMA large programs. We fit the emission line fluxes from NIRSpec and the broad-band photometry from NIRCam with Prospector, using both spatially integrated emission and $\sim0.6$ kpc pixel-by-pixel measurements. We derive the stellar-to-nebular dust attenuation ratio ($f=E(B-V)_{\mathrm{star}}/E(B-V)_{\mathrm{neb}}$) from the SED fits and the Balmer decrement with H$\alpha$ and H$\beta$. Although individual galaxies show large scatter, the best-fit value is $f = 0.51^{+0.04}_{-0.03}$, slightly higher than that measured for local starburst galaxies. We find weak correlations of $f$ with galaxy properties, increasing with higher specific star-formation rates, younger stellar ages, and more recent star-formation. For the range of $E(B-V)_{\mathrm{star}} = 0.009-0.15$ mag for in our sample, assuming $f = 1$ (often adopted in high-redshift studies) instead of $f = 0.51$ underestimate line luminosities and ionizing photon production efficiency $\xi_\text{ion}$ by $\sim3-36\%$ and $\sim4-46\%$, respectively. We also find that the total stellar masses estimated from spatially-integrated SED fits with a delayed-$\tau$ star-formation histories are systematically smaller than the sum of pixel-by-pixel SED fits, with a median offset of $\sim 0.26$ dex, likely because the integrated fits are biased toward luminous young stellar populations.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.18382-b31b1b.svg)](https://arxiv.org/abs/2510.18382) | **Acceleration of planetary migration: Resonance crossing and planetesimal ring**  |
|| H. Li, L.-Y. Zhou, <mark>X. Zhang</mark> |
|*Appeared on*| *2025-10-22*|
|*Comments*| *11 pages, 6 figures. Accepted by A&A*|
|**Abstract**|            Planetary migration is a crucial stage in the early solar system, explaining many observational phenomena and providing constraints on details related to the solar system's origins. This paper aims to investigate the acceleration during planetary migration in detail using numerical simulations, delving deeper into the early solar system's preserved information. We confirm that planetary migration is a positive feedback process: the faster the migration, the more efficient the consumption of planetesimals; once the migration slows down, Neptune clears the surrounding space, making further migration more difficult to sustain. Quantitatively, a tenfold increase in migration rate corresponds to an approximately 30% reduction in the mass of planetesimals consumed to increase per unit angular momentum of Neptune. We also find that Neptune's final position is correlated with the initial surface density of planetesimals at that location, suggesting that the disk density at 30au was approximately 0.009$M_{\oplus}/au^2$ in the early solar system. Two mechanisms that can accelerate planetary migration are identified: the first is MMR between Uranus and Neptune. Migration acceleration will be triggered whenever these two giant planets cross their major MMR. The second mechanism is the ring structure within the planetesimal disk, as the higher planetesimal density in this region can provide the material support necessary for migration acceleration. Our research indicates that Neptune in the current solar system occupies a relatively delicate position. In case Neptune crossed the 1:2 MMR with Uranus, it could have migrated to a much more distant location. Therefore, under the influence of the positive feedback mechanism, the evolution of the solar system to its current configuration might be a stochastic outcome rather than an inevitable consequence.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.18531-b31b1b.svg)](https://arxiv.org/abs/2510.18531) | **Improved thermonuclear rate of $^{42}$Ti($p$,$γ$)$^{43}$V and its astrophysical implication in rp-process**  |
|| S. Q. Hou, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-10-22*|
|*Comments*| *7 pages, 5 figures, published in Astronomy & Astrophysics*|
|**Abstract**|            Accurate $^{42}$Ti($p$,$\gamma$)$^{43}$V reaction rates are crucial for understanding the nucleosynthesis path of the rapid capture process (rp-process) that occurs in X-ray bursts. We aim to improve the thermonuclear rates of $^{42}$Ti($p$,$\gamma$)$^{43}$V based on more complete resonance information and accurate direct component, together with the recently released nuclear masses data. We reevaluated the $^{42}$Ti($p$,$\gamma$)$^{43}$V rate by the sum of the isolated resonance contribution instead of the Hauser-Feshbach statistical model. A Monte Carlo method is used to derive the uncertainties of new rates. The nucleosynthesis simulations are performed via the NuGrid post-processing code ppn. The new rates differ from previous estimations because of using a series of updated resonance parameters and direct S-factor. Compared with the previous results from Hauser-Feshbach statistical model, which assumes compound nucleus $^{43}$V with a sufficiently high-level density in the energy region of astrophysical interest, differences exist over the entire temperature region of rp-process interest, even up to 4 orders of magnitude. Using a trajectory with a peak temperature of 1.95$\times$10$^9$ K, we perform the rp-process nucleosynthesis simulations to investigate the impact of the new rates. Our calculations show that the adoption of the new forward and reverse rates result in abundance variations for Sc and Ca by 128\% and 49\% respectively compared to the case using statistical model rates. On the other hand, the overall abundance pattern is not significantly affected. The results of using new rates also confirm that the rp-process path does not bypass the isotope $^{43}$V. It is found that the Hauser-Feshbach statistical model is inappropriate to the reaction rate evaluation for $^{42}$Ti($p$,$\gamma$)$^{43}$V.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.18227-b31b1b.svg)](https://arxiv.org/abs/2510.18227) | **Milky Way Mass Through Escape Velocity Curve from LAMOST K Giants**  |
|| <mark>Y. Wu</mark>, et al. |
|*Appeared on*| *2025-10-22*|
|*Comments*| *13 pages, 8 figures, published in ApJ*|
|**Abstract**|            Escape velocity has long been used to constrain the mass of the Dark Matter (DM) halo in the Milky Way (MW). Here we present a study of the escape velocity curve using a sample of high-velocity K giants with full 6D phase-space information and relatively good quality, selected from LAMOST DR8 and cross-matched with Gaia DR3. To expand the high-velocity stars to larger distances, we used radius-dependent criteria of total velocity, that is, $v_{\rm GC}>300\;\rm{km s^{-1}}$ for the solar neighborhood; $v_{\rm GC}>v_{\rm min}\sim0.6\times v_{\rm esc}(r_{\rm GC})$ for outer region. We also selected halo stars based on $v_{\phi}-{\rm [Fe/H]}$ information to ensure that the sample is isotropic. We modeled the velocity distribution with traditional power-law models to determine the escape velocity in each radial bin. For the first time, we have directly measured a relatively continuous escape velocity curve that can extend to Galactocentric radii of $\sim 50$ kpc, finding a decline in agreement with previous studies. The escape velocity at the solar position yielded by our measurements is $523.74^{+12.83}_{-13.47}\;\rm{km s^{-1}}$. Combined with the local circular velocity, we estimated the mass of the MW assuming a Navarro-Frenk-White DM profile, which resulted in a total mass of $M_{200,\;\rm{total}}=0.90_{-0.07}^{+0.06}\times 10^{12}\;M_{\odot}$, with a concentration of $c_{200}=13.47_{-1.70}^{+1.85}$. The small uncertainty implies that including the escape velocities beyond the solar neighborhood can result in a more precise mass estimate. Our derived MW mass is consistent with some recent studies using the escape velocity as well as other tracers, which may support a lower mass of the DM halo than in the past.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2510.17957/./collage_panels_new_CV.png', 'tmp_2510.17957/./pd_classes.png', 'tmp_2510.17957/./Teff_Mdot_vs_Porb21.png']
copying  tmp_2510.17957/./collage_panels_new_CV.png to _build/html/
copying  tmp_2510.17957/./pd_classes.png to _build/html/
copying  tmp_2510.17957/./Teff_Mdot_vs_Porb21.png to _build/html/
exported in  _build/html/2510.17957.md
    + _build/html/tmp_2510.17957/./collage_panels_new_CV.png
    + _build/html/tmp_2510.17957/./pd_classes.png
    + _build/html/tmp_2510.17957/./Teff_Mdot_vs_Porb21.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Msun}{\ensuremath{M_\odot}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\msun}{M_\odot}$
$\newcommand{\arraystretch}{0.9}$</div>



<div id="title">

# The Silent Majority: The Interacting Post-Common-Envelope Binaries Underlying Cataclysmic Variables

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.17957-b31b1b.svg)](https://arxiv.org/abs/2510.17957)<mark>Appeared on: 2025-10-22</mark> - 

</div>
<div id="authors">

Y. M. Shani, et al. -- incl., <mark>S. Shahaf</mark>, <mark>J. Li</mark>, <mark>H.-W. Rix</mark>

</div>
<div id="abstract">

**Abstract:** We analyze the orbital period distribution of post-common-envelope white-dwarf-main-sequence (WDMS) binaries by cross-matching the new spectroscopic Gaia DR3 WDMS catalog with TESS light curves, and applying a uniform periodicity search and vetting pipeline. We identify 107 periodic systems, including 74 eclipsing binaries (32 new) and 33 binaries exhibiting only sinusoidal variations. Injection-recovery tests and a forward detectability model yield a completeness-corrected distribution that is well-described by a two-component function: a log-period Gaussian peaking at $P_{\rm orb} \approx 4.1 $ h with $\sigma \approx 1.8$ h, plus a rising component that begins near $P_{\rm orb}\approx12.9$ h. We refer to this extended component as the long-period tail. It consists exclusively of detached non-interacting post-common-envelope binaries (PCEBs) that likely emerged from the common envelope and have not yet initiated mass transfer. In contrast, the short-period Gaussian is dominated by interacting or near-contact systems (including 22 known cataclysmic variables), consistent with high Roche-lobe filling factors. From the completeness-corrected distribution we infer that $29.8\%\pm4.5\%$ of the spatially unresolved WDMSs in our parent catalog are close PCEBs. Binary population synthesis models with high common-envelope efficiencies overproduce long-period systems and fail to reproduce the sharp peak, whereas lower efficiencies ( $\alpha\lambda \leq 0.3$ ) match the peak more closely, yet still underpredict the tail. Our results hint at a large, currently under-classified reservoir of pre-cataclysmic variables and weakly accreting binaries, and provide new constraints on common-envelope physics.

</div>

<div id="div_fig1">

<img src="tmp_2510.17957/./collage_panels_new_CV.png" alt="Fig10" width="100%"/>

**Figure 10. -** Phase-folded (gray) and binned (black) TESS light curves of the known CVs in our sample. (*fig:ecl1*)

</div>
<div id="div_fig2">

<img src="tmp_2510.17957/./pd_classes.png" alt="Fig7" width="100%"/>

**Figure 7. -** Stacked histogram of the orbital period distribution (in log hours) for three sub-classes:
    eclipsing PCEBs (light gray, N = 52), CVs (medium gray, N = 22),  and sinusoidal variables (dark gray, N = 33). Vertical dashed lines mark the minimum observed CV period ($\approx$  80 min ) and the canonical lower ($\sim2$ h) and upper ($\sim3$ h) boundaries of the CV "period gap". (*fig:ellipsoidals_period_distribution*)

</div>
<div id="div_fig3">

<img src="tmp_2510.17957/./Teff_Mdot_vs_Porb21.png" alt="Fig17" width="100%"/>

**Figure 17. -** Comparison of our eclipsing WDMS systems (52 unclassified PCEBs + 22 CVs; sinusoidal variables excluded) to the UV-calibrated CV locus from the HST+Gaia sample of [Pala, Gänsicke and Belloni (2022)](). _Top_: WD effective temperature versus orbital period. _Bottom_: equivalent long-term accretion rate $\langle\dot M\rangle$(inferred from the WD's $T_{\rm eff}$ following the UV calibration) versus orbital period. Vertical dashed lines mark the minimum observed CV period ($\approx$  80 min ) and the canonical lower ($\sim2$ h) and upper ($\sim3$ h) boundaries of the CV "period gap". Unclassified eclipsers at $P_\text{orb} \approx 3  -  6$ h lie on the same $T_{\rm eff}  -  P_{\rm orb}$ sequence as CVs above the period gap, indicating that many systems are near, or already interacting, even without a CV classification. (*fig:temp_period*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.17957"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

115  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
